In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import warnings
from IPython.display import Image
from causalnex.plots import plot_structure
from causalnex.structure import StructureModel
from causalnex.structure.notears import from_pandas
warnings.filterwarnings("ignore")  # silence warnings
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
df=pd.read_csv(r'C:\Users\adaml\Python\Fairness Systems\causal.csv')

In [5]:
df.shape

(48842, 14)

In [18]:
def preprocess_data(df):
    # Replace "?" with "unknown"
    df.replace("?", "unknown", inplace=True)

    # Remove blank spaces
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Change 'race' to binary values first
    non_numeric_columns = list(df.select_dtypes(exclude=[np.number]).columns)
    le = LabelEncoder()

    for col in non_numeric_columns:
        df[col] = le.fit_transform(df[col])
    print(non_numeric_columns)


    return df
df = preprocess_data(df)


[]


In [19]:
import warnings
from causalnex.structure import StructureModel
from causalnex.structure.notears import from_pandas
warnings.filterwarnings("ignore")  # silence warnings
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
sm = StructureModel()

sm = from_pandas(df)

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
# %% 
viz.toggle_physics(False)
viz.show("01_fully_connected.html")

01_fully_connected.html


In [22]:
sm.remove_edges_below_threshold(1.0)
viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.show("01_thresholded.html")

01_thresholded.html


In [23]:

all_node_attributes = {
    "font": {
        "color": "#FFFFFFD9",
        "face": "Helvetica",
        "size": 20,
    },
    "shape": "box",
    "size": 15,
    "borderWidth": 2,
    "color": {
        "border": "#4a90e2d9",
        "background": "#001521"
    },
    "mass": 3
}

node_attributes = {
    "income": {
        "color": {
            "background": "#ff0000",
            "highlight": {
                "background": "#ffcccc",
                "border": "#cce0ff"
            }
        },
        "shape": "circle",
        "size": 50,

        "font": {
            "color": "#000000",
        },
        "fixed": {
            "y": True
        }
    },
    "sex": {
        "color": {
            "background": "#FFD700",
            "highlight": {
                "background": "#ffcccc",
                "border": "#cce0ff"
            }
        },
        "shape": "hexagon",
        "size": 20,
        "fixed": {
            "x": True
        }
    },
     "age": {
        "color": {
            "background": "#FFD700",
            "highlight": {
                "background": "#ffcccc",
                "border": "#cce0ff"
            }
        },
        "shape": "hexagon",
        "size": 20,
        "fixed": {
            "x": True
        }
    }
}

viz2 = plot_structure(
    sm, all_node_attributes=all_node_attributes, node_attributes=node_attributes
)
viz2.show('02_new_node.html')

02_new_node.html


In [24]:
import json
from causalnex.plots import EDGE_STYLE, NODE_STYLE

opt = {
    "layout": {
        "hierarchical": {
        "enabled": True,
        "direction": "LR", #LR means that the hierarchy is displayed left to right
        "sortMethod": "directed"
        }
    }
}

viz5 = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz5.set_options(options=json.dumps(opt))
viz5.show('03_new_hierarchical.html')

03_new_hierarchical.html


Bayesian Network

In [25]:
import networkx as nx

nx.drawing.nx_pydot.write_dot(sm, 'graph.dot')

In [26]:
from causalnex.network import BayesianNetwork

bn = BayesianNetwork(sm)

In [27]:
data = pd.read_csv(r'C:\Users\adaml\Python\Fairness Systems\causal.csv')

# Discretizing age
age_map = {
    range(0, 30): "Young",
    range(30, 61): "Middle-aged",
    range(61, 150): "Senior"
}
data["age"] = data["age"].apply(lambda x: next((v for k, v in age_map.items() if x in k), x))

# Discretizing edunum
edunum_map = {
    range(0, 10): "Low-education",
    range(10, 13): "Medium-education",
    range(13, 25): "High-education"
}
data["edunum"] = data["edunum"].apply(lambda x: next((v for k, v in edunum_map.items() if x in k), x))

# Discretizing capitalgain and capitalloss
capitalgain_median = data["capitalgain"].median()
capitalloss_median = data["capitalloss"].median()

data["capitalgain"] = data["capitalgain"].apply(lambda x: "No-gain" if x == 0 else ("Low-gain" if x < capitalgain_median else "High-gain"))
data["capitalloss"] = data["capitalloss"].apply(lambda x: "No-loss" if x == 0 else ("Low-loss" if x < capitalloss_median else "High-loss"))

# Discretizing hoursperweek
hours_map = {
    range(0, 35): "Part-time",
    range(35, 51): "Full-time",
    range(51, 150): "Overtime"
}
data["hoursperweek"] = data["hoursperweek"].apply(lambda x: next((v for k, v in hours_map.items() if x in k), x))

# Displaying the updated data
data.head()


,age,workclass,education,edunum,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,country,income,predicted income
0,Middle-aged,State-gov,Bachelors,High-education,Never-married,Adm-clerical,Not-in-family,White,Male,High-gain,No-loss,Full-time,United-States,<=50K,>50K
1,Middle-aged,Self-emp-not-inc,Bachelors,High-education,Married-civ-spouse,Exec-managerial,Husband,White,Male,No-gain,No-loss,Part-time,United-States,<=50K,>50K
2,Middle-aged,Private,HS-grad,Low-education,Divorced,Handlers-cleaners,Not-in-family,White,Male,No-gain,No-loss,Full-time,United-States,<=50K,<=50K
3,Middle-aged,Private,11th,Low-education,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,No-gain,No-loss,Full-time,United-States,<=50K,<=50K
4,Young,Private,Bachelors,High-education,Married-civ-spouse,Prof-specialty,Wife,Black,Female,No-gain,No-loss,Full-time,Cuba,<=50K,>50K


In [28]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, train_size=0.9, test_size=0.1, random_state=7)

In [29]:
bn = bn.fit_node_states(data)

In [30]:
bn = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")

MemoryError: Unable to allocate 1.28 GiB for an array with shape (84672, 16200) and data type int8

In [3]:
from pgmpy.models import BayesianModel
# from pgmpy.inference import BayesianModelSampling
from pgmpy.estimators import ParameterEstimator, MaximumLikelihoodEstimator, BayesianEstimator
from pgmpy.utils import get_example_model

# Construct the Bayesian Network from edges
edges = [
    ("age", "edunum"),
    ("age", "capitalgain"),
    ("workclass", "education"),
    ("workclass", "capitalgain"),
    ("workclass", "capitalloss"),
    ("education", "race"),
    ("education", "capitalgain"),
    ("education", "country"),
    ("edunum", "capitalgain"),
    ("edunum", "capitalloss"),
    ("maritalstatus", "capitalgain"),
    ("maritalstatus", "capitalloss"),
    ("maritalstatus", "hoursperweek"),
    ("occupation", "age"),
    ("occupation", "workclass"),
    ("occupation", "education"),
    ("occupation", "maritalstatus"),
    ("occupation", "relationship"),
    ("occupation", "capitalgain"),
    ("occupation", "hoursperweek"),
    ("occupation", "country"),
    ("relationship", "education"),
    ("relationship", "capitalgain"),
    ("relationship", "capitalloss"),
    ("relationship", "country"),
    ("race", "capitalgain"),
    ("sex", "capitalgain"),
    ("sex", "capitalloss"),
    ("capitalloss", "capitalgain"),
    ("hoursperweek", "capitalgain"),
    ("country", "edunum"),
    ("country", "race"),
    ("country", "capitalgain"),
    ("income", "capitalgain"),
    ("income", "capitalloss"),
    ("income", "hoursperweek"),
    ("predicted income", "capitalgain"),
    ("predicted income", "capitalloss")
]

bn = BayesianModel(edges)

# Check for cycles
is_valid = bn.check_model()

is_valid


ValueError: No CPD associated with age